# Semantic Network

In [76]:
import pprint
from rdflib import Graph, URIRef, Literal,RDF,RDFS
from rdflib.namespace import FOAF

import numpy as np
import pandas as pd


In [77]:
class Node:
    Poi_ID = ''
    name = ''
    Uri = ''
    Pref = 0
    Activate = 0.0
    FactorCtx = {}
    Context = 0
    conf = 0.0
    alfa = 0.1
    
    def __init__(self,Uri):
        self.Uri = Uri
    
    def updateContext(self,nContextAncestros):
        sum_ = 0
        sum_factor = 0
        for key, value in self.FactorCtx.items():
            sum_factor = self.FactorCtx[key]
            sum_ += 1
        if sum_factor== 0:
            self.Context = 0.1
        else:
            self.Context = sum_factor/sum_
            
            
        sum_contex = 0
        for pref_ in nContextAncestros:
            sum_contex += 1
        self.Context = (sum_contex + self.Context)/ (len(nContextAncestros) + 1)
        
    def updatePref(self,nprefAncestros):
        sum_ = 0
        sum_pref = 0
        for pref_ in nprefAncestros:
            pref_ += 1
            sum_ += 1
        conf = conf - 0.1
        self.Pref = sum_pref/sum_-conf
        
        
    def pprint_(self):
        print('Uri', self.Uri)
        print('Pref',self.Pref)
        print('Poi_ID',self.Poi_ID)
        print('Activate',self.Activate)
        print('FactorCtx',self.FactorCtx)
        print('Context',self.Context)
        print('Conf',self.conf)
            

In [78]:
def format_str(str_):
        idx = str_.find(':')
        return str_[idx+1:]


In [79]:
def format_str_num(str_):
        idx = str_.find('#')
        return str_[idx+1:]

In [80]:
class SemanticNetwork:
    uidUser = ''
    NodeObjPOI = {}
    graph = Graph()
    classPr = []
    Subclass = {}
    name_id = {}
    ChildenOf ={}
    
    def __init__(self,uidUser):
        self.uidUser = uidUser
        
    def updatePOIs(self, arrayUri):
        cont = 1
        for uri in arrayUri:
            #print("->",format_str(uri))
            poiUri = URIRef(uri)
            node_= Node(uri)
            node_.name = format_str_num(uri)
            node_.conf = 1
            node_.Poi_ID = f"{cont}p"
            #self.graph.parse(poiUri)
            self.NodeObjPOI[node_.Poi_ID] = node_
            self.name_id[format_str_num(uri)] = f"{cont}p"
            cont+=1
        path = '/home/yessica/Documentos/2021/ProyectoTesis/SR/SRTourism/ontology.ttl'
        self.graph.parse(path,format = 'ttl')
        
        
    def printGraph(self):
        graphSerialize = self.graph.serialize(format="xml")

        #print(g.serialize())

        for subj, pred, obj in self.graph:

            print('->subj ', subj)
            print('->pred ', pred)
            print('->obj ', obj)
            print('\n')
            
    def updatePref(self, arrayPref):
        for key,value in arrayPref.items():
            self.NodeObjPOI[key].Pref = value
            
    def updateContext(self, arrayContext):
        for key,context in arrayContext.items():
            self.NodeObjPOI[key].FactorCtx = context
        
    def spreadingActivation(self):
        for subj, pred, obj in self.graph:
            for POIsk,POINode in self.NodeObjPOI.items():
                uri = POINode.Uri
                if subj == URIRef(uri) and type(obj)==URIRef:
                    subjStr = subj.n3(self.graph.namespace_manager)
                    subj = format_str(subjStr)
                    
                    predStr = pred.n3(self.graph.namespace_manager)
                    pred = format_str(predStr)
                    
                    objStr = obj.n3(self.graph.namespace_manager)
                    obj = format_str(objStr)
                    
                    if pred == 'type':
                        self.classPr.append(subj) #subj self.name_id[subj]
                        
                    elif pred == 'subClassOf':
                        if obj not in self.Subclass:
                            self.Subclass[obj]=[]
                        self.Subclass[obj].append(subj)
                        
                        if subj not in self.ChildenOf:
                            self.ChildenOf[subj]=[]
                        self.ChildenOf[subj].append(obj)
                        
    def propagationPref(self,node_, nodeAncesters):
        sumPrefc = 0
        confC = 0 
        if nodeAncesters.Pref == 0 and node_.Pref>0:
            sumPrefc = node_.Pref * node_.conf
            confC += node_.conf
            nodeAncesters.Pref = sumPrefc/confC
            nodeAncesters.conf = ((node_.conf - node_.alfa )+ nodeAncesters.conf)/2
    
            
    def calculateActivate(self, node_, arrayContext):
        context = arrayContext
        factCtx = node_.FactorCtx
        sumFactor = 0
        nfactor = 0

        for key in factCtx.keys():
            if context[key]== 'any':
                sumFactor += 1
                nfactor += 1
            elif context[key] in factCtx[key]:
                sumFactor += 2
                nfactor += 1
            else:
                sumFactor += 0
                nfactor += 1
            #print('sumFactor :',sumFactor)

        node_.Activate = sumFactor/nfactor
        
    def spreadingActivationPropagation(self):
        for key in self.Subclass.keys():
            if key not in self.name_id:
                idPlace =  f"{len(semanticNet.name_id)+1}p"
                self.name_id[key] = idPlace
                node_= Node('')
                node_.name = key
                node_.Poi_ID = idPlace
                self.NodeObjPOI[node_.Poi_ID] = node_
        
        for key in self.ChildenOf:
            id__ = self.name_id[key]
            node_ = self.NodeObjPOI[id__]
            ancestors = []
            
            arrayContext = {
                    'weather':'rainy',
                    'date':'any',
                    'time':'afternoon',
                    'summer':'any'
                    }
            if node_.Activate <= 0:
                self.calculateActivate( node_, arrayContext)

            for father in self.ChildenOf[key]:
                id_ = self.name_id[father]
                node = self.NodeObjPOI[id_]
                self.propagationPref(node_,node)
                node.Activate = (node_.Activate + node.Activate)/2
    
            
        
                        
                        

In [81]:
semanticNet = SemanticNetwork('id1')
arrUri = [  "https://www.datatourisme.gouv.fr/ontology/core#Museum",
            "https://www.datatourisme.gouv.fr/ontology/core#InterpretationCentre",
            "https://www.datatourisme.gouv.fr/ontology/core#Library",
            "https://www.datatourisme.gouv.fr/ontology/core#ParkAndGarden",
            "https://www.datatourisme.gouv.fr/ontology/core#ArcheologicalSite",
            "https://www.datatourisme.gouv.fr/ontology/core#ReligiousSite",
            "https://www.datatourisme.gouv.fr/ontology/core#RemarkableBuilding",
            "https://www.datatourisme.gouv.fr/ontology/core#CityHeritage",
            "https://www.datatourisme.gouv.fr/ontology/core#DefenceSite",
            "https://www.datatourisme.gouv.fr/ontology/core#RemembranceSite",
            "https://www.datatourisme.gouv.fr/ontology/core#TechnicalHeritage",
            "https://www.datatourisme.gouv.fr/ontology/core#FoodEstablishment",
            "https://www.datatourisme.gouv.fr/ontology/core#NaturalHeritage",
            "https://www.datatourisme.gouv.fr/ontology/core#SportsAndLeisurePlace",
            "https://www.datatourisme.gouv.fr/ontology/core#Store"]


In [82]:
semanticNet.updatePOIs(arrUri)
arrPOIs = semanticNet.NodeObjPOI

In [83]:
arrayPref = {'1p':1.00,      '2p':1.394117647, '3p': 1.029411765, '4p':1.082352941, '5p':1.488235294, 
             '6p':1.064705882,'7p':1.517647059, '8p': 0.7882352941,'9p':0.1941176471,'10p': 0.8705882353,
             '11p':1.311764706,'12p':0.8294117647,'13p':1.029411765,'14p': 1.423529412,'15p': 1.029411765
            }

semanticNet.updatePref(arrayPref)

In [84]:
arrayContext = {'1p':{
                    'weather':['rainy'],
                    'date':['workday'],
                    'time':['morning','afternoon'],
                    'summer':['any']   
                    },     
                '2p':{
                    'weather':['rainy'],
                    'date':['workday','weekend'],
                    'time':['morning','afternoon'],
                    'summer':['any']   
                    }, 
                '3p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] }, 
                '4p':{
                    'weather':['rainy'],
                    'date':['workday','weekend'],
                    'time':['morning','afternoon'],
                    'summer':['any'] },  
                '5p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] }, 
                '6p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
             '7p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] }, 
             '8p': {
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
            '9p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
            '10p': {
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
            '11p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
            '12p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
            '13p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
            '14p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
            '15p': {
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] }
            }

In [85]:
semanticNet.updateContext(arrayContext)

In [86]:
semanticNet.spreadingActivation()


In [87]:
semanticNet.spreadingActivationPropagation()

# Extraer la relacion de pref y activacion(context) de un Id_User y POIs 

In [88]:
def relation(semanticNet):
    relation = []
    w1 = 0.3
    w2 = 0.3
    for PoiId in semanticNet.NodeObjPOI:
        
        rating = arrPOIs[PoiId].Pref*w1 + arrPOIs[PoiId].Activate*w2
        
        relation.append([semanticNet.uidUser,
                 PoiId,
                 arrPOIs[PoiId].Pref,
                 arrPOIs[PoiId].Activate,
                 rating
                ])
        
    return relation

    

In [89]:
relation = relation(semanticNet)
relation

array = np.array(relation)
df = pd.DataFrame(array, columns = ['Id','POIid','Pref','Activate','rating'])

print(df)

     Id POIid          Pref    Activate               rating
0   id1    1p           1.0         1.5                 0.75
1   id1    2p   1.394117647         1.5         0.8682352941
2   id1    3p   1.029411765         1.0         0.6088235295
3   id1    4p   1.082352941         1.5   0.7747058822999999
4   id1    5p   1.488235294         1.0         0.7464705882
5   id1    6p   1.064705882         1.0   0.6194117645999999
6   id1    7p   1.517647059         1.0   0.7552941176999999
7   id1    8p  0.7882352941         1.0        0.53647058823
8   id1    9p  0.1941176471         1.0        0.35823529413
9   id1   10p  0.8705882353         1.0        0.56117647059
10  id1   11p   1.311764706         1.0   0.6935294117999999
11  id1   12p  0.8294117647         1.0        0.54882352941
12  id1   13p   1.029411765         1.0         0.6088235295
13  id1   14p   1.423529412         1.0         0.7270588236
14  id1   15p   1.029411765         1.0         0.6088235295
15  id1   16p   1.029411

In [91]:
df.to_csv('../mod_recomendacion/datasetPropuesta/test_individual.csv') 